In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\Users\tulsi\AppData\Local\Temp\ipykernel_16692\1321570591.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [3]:
df =pd.read_csv("E:/NMIMS work/Project/lie-detection/code/previous-MLcode/lie_dataset.csv")
df.head()

,lipsx,lipsy,left_browx,left_browy,right_browx,right_browy,body0x,body0y,body1x,body1y,...,body7y,face0x,face0y,face1x,face1y,face2x,face2y,face3x,face3y,TRUTH
0,-0.001107,0.031804,-0.053075,-0.033591,0.060648,-0.029370,0.244749,-0.003385,-0.244749,0.003385,...,0.544281,0.008663,-0.040341,-0.004295,0.063091,-0.080905,0.017946,0.079827,0.023363,1
1,-0.001794,0.030796,-0.052959,-0.032074,0.059244,-0.029025,0.246057,-0.003136,-0.246057,0.003136,...,0.544409,0.007592,-0.037885,-0.004266,0.061061,-0.079106,0.019367,0.077311,0.023393,1
2,-0.002718,0.031074,-0.054027,-0.032401,0.058203,-0.028548,0.246974,-0.002954,-0.246974,0.002954,...,0.545489,0.006395,-0.037179,-0.006021,0.060508,-0.081213,0.019012,0.075062,0.023403,1
3,-0.002734,0.031018,-0.053086,-0.032754,0.059224,-0.028536,0.247235,-0.002925,-0.247235,0.002925,...,0.545792,0.007735,-0.037539,-0.006494,0.060074,-0.080609,0.018408,0.075682,0.023359,1
4,-0.002798,0.030957,-0.052412,-0.032349,0.058886,-0.028198,0.247395,-0.002847,-0.247395,0.002847,...,0.545749,0.007852,-0.037106,-0.006559,0.060057,-0.080242,0.018465,0.075385,0.023527,1


In [4]:
X = df.copy()
y = X.pop("TRUTH")

input_shape = X.shape[1]
input_shape

30

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.6)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint ## To save best accuracy or loss as a checkpoint

In [8]:
#X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
#X_valid = np.reshape(X_valid, (X_valid.shape[0],1,X_valid.shape[1]))
#X_train.shape

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=X_train.shape, return_sequences=True))
model.add(Dropout(0.2))
#model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
#model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
#model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

In [10]:
opt = tf.keras.optimizers.Adam()

# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [11]:
print(y_train.shape)
print(X_train.shape)
print(X_valid.shape)
print(y_valid.shape)


(189,)
(189, 30)
(126, 30)
(126,)


In [56]:
history = model.fit(
    X_train, y_train,
    batch_size=64,
    epochs=100,
    validation_data=(X_valid, y_valid)
    #callbacks=[tensorboard, checkpoint],
)

Epoch 1/100


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In